<a href="https://colab.research.google.com/github/MSBX5420/team-mount-massive/blob/master/Abbie_Simple_Top_5_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

Load in data and create a copy.

In [7]:
url = 'https://raw.githubusercontent.com/MSBX5420/team-mount-massive/master/sample_movies_single.csv'
movies = pd.read_csv(url)
movies2 = movies.copy()


,titleId,ordering,title,region,language,types,attributes,isOriginalTitle,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000012,7,L'arrivée d'un train à La Ciotat,\N,\N,original,\N,1,tt0000012,short,The Arrival of a Train,L'arrivée d'un train à La Ciotat,0.0,1896,\N,1,"Action,Documentary,Short",7.4,9435.0
1,tt0000582,4,Ben Hur,\N,\N,original,\N,1,tt0000582,short,Ben Hur,Ben Hur,0.0,1907,\N,15,"Drama,Short",5.1,555.0
2,tt0001013,1,The Red Man's View,\N,\N,original,\N,1,tt0001013,short,The Red Man's View,The Red Man's View,0.0,1909,\N,14,"Short,Western",5.9,283.0
3,tt0011545,8,Oranges and Lemons,\N,\N,original,\N,1,tt0011545,short,Oranges and Lemons,Oranges and Lemons,0.0,1923,\N,12,"Comedy,Short",5.8,367.0
4,tt0013951,7,The Covered Wagon,\N,\N,original,\N,1,tt0013951,movie,The Covered Wagon,The Covered Wagon,0.0,1923,\N,98,"Adventure,Romance,Western",6.5,470.0


Create a new Weighted Rating Column in data copy using formula cited to be from IMDB on Kaggle (though IMDB says it keeps its weighted rating formula private to maintain efficacy). This formula takes number of votes for the movie, the average rating over all movies, the average rating for the movie, and an inputed value representing the number of votes requred to make the list (I chose movies over the 10th percentile since we already created a cutoff in the data cleaning process).  

In [68]:
movies2["WeightedRating"] = np.nan

for i in range(0,len(movies2)) :
  v = movies2.numVotes[i]
  m = np.percentile(movies2.averageRating, q = .10)
  C = movies2['averageRating'].mean()
  R = movies2.averageRating[i]
  movies2.WeightedRating[i] = (v/(v+m) * R) + (m/(m+v) * C)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,titleId,ordering,title,region,language,types,attributes,isOriginalTitle,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,WeightedRating
0,tt0000012,7,L'arrivée d'un train à La Ciotat,\N,\N,original,\N,1,tt0000012,short,The Arrival of a Train,L'arrivée d'un train à La Ciotat,0.0,1896,\N,1,"Action,Documentary,Short",7.4,9435.0,7.399816
1,tt0000582,4,Ben Hur,\N,\N,original,\N,1,tt0000582,short,Ben Hur,Ben Hur,0.0,1907,\N,15,"Drama,Short",5.1,555.0,5.103897
2,tt0001013,1,The Red Man's View,\N,\N,original,\N,1,tt0001013,short,The Red Man's View,The Red Man's View,0.0,1909,\N,14,"Short,Western",5.9,283.0,5.902844
3,tt0011545,8,Oranges and Lemons,\N,\N,original,\N,1,tt0011545,short,Oranges and Lemons,Oranges and Lemons,0.0,1923,\N,12,"Comedy,Short",5.8,367.0,5.802657
4,tt0013951,7,The Covered Wagon,\N,\N,original,\N,1,tt0013951,movie,The Covered Wagon,The Covered Wagon,0.0,1923,\N,98,"Adventure,Romance,Western",6.5,470.0,6.499554


Create a function that takes three parameters: genre, type, and age of film. This function cycles through films meeting the criteria, and produces a top 5 list based on highest weighted rating. 

In [0]:
def recommend(genrechoice, typechoice, agechoice = 0):
  PossMovie = movies2[movies2['genres'].str.contains(genrechoice)]
  PossMovie = PossMovie[PossMovie['titleType'].str.contains(typechoice)]
  if (agechoice == "Old" ):
    PossMovie = PossMovie[PossMovie.startYear <= 2000]
  if (agechoice == "New" ):
    PossMovie = PossMovie[PossMovie.startYear > 2000]
  else: 
    PossMovie = PossMovie
  Ordered = PossMovie.sort_values(by="WeightedRating", ascending = False) 
  Ordered = Ordered[['title', 'genres', 'titleType', 'startYear', 'WeightedRating']]
  Top5 = Ordered[:5]
  return Top5

Test function:

In [74]:
recommend('Comedy', 'movie', 'New')

,title,genres,titleType,startYear,WeightedRating
769,Marmoulak,"Comedy,Drama",movie,2004,8.499704
663,Dil Chahta Hai,"Comedy,Drama,Romance",movie,2001,8.099953
1135,Guardians of the Galaxy,"Action,Adventure,Comedy",movie,2014,7.999997
1248,Zootopia,"Adventure,Animation,Comedy",movie,2016,7.999993
1337,En man som heter Ove,"Comedy,Drama,Romance",movie,2015,7.699945
